In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Load Data

In [6]:
import pandas as pd
df = pd.read_json("/content/drive/MyDrive/dataset/Sequence NLP/Sarcasm_Headlines_Dataset.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


## Drop `article_link` in the  dataset

In [7]:
df = df.drop(['article_link'], axis=1)
df.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


## Length of each headline

In [8]:
df['len'] = df['headline'].apply(lambda x: len(x.split(" ")))
df.head()

,is_sarcastic,headline,len
0,1,thirtysomething scientists unveil doomsday clo...,8
1,0,dem rep. totally nails why congress is falling...,13
2,0,eat your veggies: 9 deliciously different recipes,7
3,1,inclement weather prevents liar from getting t...,8
4,1,mother comes pretty close to using word 'strea...,9


In [11]:
!pip uninstall tensorflow

In [12]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3 MB 12 kB/s 
     |████████████████████████████████| 3.8 MB 18.6 MB/s 
     |████████████████████████████████| 449 kB 31.7 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=78c3ee03e1206954f9032b1f5eb7b000da07af2f552913b5f8d1cb7f371f1fc3
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Success

In [13]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

In [14]:
max_features = 10000
maxlen = 25
embedding_size = 200

## Apply Keras Tokenizer and get indices for words

In [15]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))
X = tokenizer.texts_to_sequences(df['headline'])
print("Number of Samples:", len(X))
print(X[0])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 28619
[354, 3166, 7473, 2643, 2, 660, 1118]
Number of Labels:  28619
1


## Vocabulary size

In [16]:
num_words = len(tokenizer.word_index) + 1
print(num_words)

30885


## Get Glove Word Embeddings

In [ ]:
glove_file = project_path + "glove.6B.zip"

In [ ]:
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

In [ ]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    embeddings[word] = embd

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
len(embeddings.values())

400000

## Model

In [ ]:
model = Sequential()
model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 5
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 21367 samples, validate on 5342 samples
Epoch 1/5
21367/21367 [==============================] - 62s 3ms/sample - loss: 0.4788 - accuracy: 0.7724 - val_loss: 0.3570 - val_accuracy: 0.8461
Epoch 2/5
21367/21367 [==============================] - 57s 3ms/sample - loss: 0.2820 - accuracy: 0.8935 - val_loss: 0.3493 - val_accuracy: 0.8647
Epoch 3/5
21367/21367 [==============================] - 58s 3ms/sample - loss: 0.1913 - accuracy: 0.9308 - val_loss: 0.3703 - val_accuracy: 0.8614
Epoch 4/5
21367/21367 [==============================] - 58s 3ms/sample - loss: 0.1285 - accuracy: 0.9543 - val_loss: 0.4733 - val_accuracy: 0.8594
Epoch 5/5
21367/21367 [==============================] - 56s 3ms/sample - loss: 0.0834 - accuracy: 0.9704 - val_loss: 0.7511 - val_accuracy: 0.8567
